In [23]:
!pip install pycoin

In [24]:
from pycoin.ecdsa.secp256k1 import secp256k1_generator
from pycoin.satoshi import der
import hashlib, secrets

In [25]:
def sha3_256Hash(msg):
    hashBytes = hashlib.sha3_256(msg.encode("utf8")).digest()
    return int.from_bytes(hashBytes, byteorder="big")

def signECDSAsecp256k1(msg, privKey):
    msgHash = sha3_256Hash(msg)
    signature = secp256k1_generator.sign(privKey, msgHash)
    return der.sigencode_der(signature[0], signature[1])

def verifyECDSAsecp256k1(msg, signature, pubKey):
    msgHash = sha3_256Hash(msg)
    r, s = der.sigdecode_der(signature)
    valid = secp256k1_generator.verify(pubKey, msgHash, (r, s))
    return valid


In [32]:

# ECDSA sign message
msg = "Tine ti poslija 10 <3"
privKey = secrets.randbelow(secp256k1_generator.order())
pubKey = secp256k1_generator * privKey
signature = signECDSAsecp256k1(msg, privKey)
print("Message:", msg)
print("Private key:", privKey)
r, s = der.sigdecode_der(signature)
print("Signature: r=" + hex(r) + ", s=" + hex(s))



Message: Tine ti poslija 10 <3
Private key: 67446254743033141283124871875225352568561396203182687730966223773612807553264
Signature: r=0xe2586804867cb1a154410dc5ce3f4a01bc41b95c529f3214ccd9b0dec61d4f4e, s=0xa9c86e78954b2b563de2f9c7e673c323e1a64c6f9246ad954f815b68ffc6c558


Signature into [DER format](https://learnmeabitcoin.com/technical/keys/signature/#der)

In [33]:
# ECDSA verify signature 
pubKey_coords = (pubKey[0], pubKey[1])
valid = verifyECDSAsecp256k1(msg, signature, pubKey)
print("\nMessage:", msg)
print("Public key: (" + hex(pubKey_coords[0]) + ", " + hex(pubKey_coords[1]) + ")")
print("Signature valid?", valid)


Message: Tine ti poslija 10 <3
Public key: (0xce469d6c24c0709c782213fa40b5c7b8eda182298d2df7613ff70988570cc570, 0x494376e43c00ba882f832df11ab4c8e1b3c36cd6df91b19475607a9d532396b3)
Signature valid? True


In [30]:
# ECDSA verify tampered signature 
msg = "Tampered message"
valid = verifyECDSAsecp256k1(msg, signature, pubKey)
print("\nMessage:", msg)
print("Signature (tampered msg) valid?", valid)


Message: Tampered message
Signature (tampered msg) valid? False


# ECDHA

a = 25
b = 24
p = 97

[echd by hand](https://andrea.corbellini.name/ecc/interactive/modk-mul.html)

# Elliptic Curves 
The mathematical backbone of ECDSA
## y² = x³ + ax + b
![Point Multiplication](pictures/elliptic-curve.png)

In summary, **public keys** and **signatures** are just points on an **elliptic curve**. If both of these points are created from the same **private key** (a large number), there will be a geometric connection between them that proves that the person who created the **signature** also created (or "owns") the **public key** too.

![Point Multiplication](pictures/point-multiply.gif)

[Eliliptic curves](https://learnmeabitcoin.com/technical/cryptography/elliptic-curve/)

## Elliptic Curve Parameters
## y² = x³ + ax + b

a = 0

b = 7

#### prime field

    p = 2 ** 256 - 2 ** 32 - 2 ** 9 - 2 ** 8 - 2 ** 7 - 2 ** 6 - 2 ** 4 - 1

#### number of points on the curve we can hit ("order")

    n = 115792089237316195423570985008687907852837564279074904382605163141518161494337

#### generator point (the starting point on the curve used for all calculations)

    G = {
      x: 55066263022277343669578718895168534326250603453777594175500187360389116729240,
      y: 32670510020758816978083085130507043184471273380659243275938904335757337482424,
    }

**Secp256k1** is just the name for one of the specific elliptic curves used in cryptography. It's short for:

- **sec** = Standard for Efficient Cryptography — A consortium that develops commercial standards for cryptography.
- **p** = Prime — A prime number is used to create the finite field.
- **256** = 256 bits — Size of the prime field used.
- **k** = Koblitz — Specific type of curve.
- **1** = First curve in this category.


![Point Multiplication](pictures/curves.png)

For this, we have to restrict ourselves to numbers in a fixed range, like in RSA. Rather than allow any value for the points on the curve, we restrict ourselves to whole numbers in a fixed range. When computing the formula for the elliptic curve (y2 = x3 + ax + b), we use the same trick of rolling over numbers when we hit the maximum. If we pick the maximum to be a prime number, the elliptic curve is called a prime curve and has excellent cryptographic properties.

Here's an example of a curve (y2 = x3 - x + 1) plotted for all numbers:
![An elliptic curve over real numbers.
](pictures/sage-elliptic-curve-real-numbers.png)

However, the actual curve used in ECDSA or ECDHE looks more like a scatter plot of points like this:
![real_ec](pictures/sage-elliptic-curve-finite-field-47.png)


This is due to the fact that the curve used in EC is over a **finite field** of whole numbers (i.e., using mod p to restrict numbers to within a certain range), and this breaks the continuous curve you're able to get when you use real numbers.

However, even though these plots look wildly different, the mathematical operations you can perform on both of these curves still work in the same way.

Of course, the **secp256k1** curve has a very large value for **p**, so it more closely resembles the graph below, except imagine there are about as many points on it as there are atoms in the universe:

![real_ec](pictures/sage-elliptic-curve-finite-field-2503.png)

With this new curve representation, you can take messages and represent them as points on the curve. You could imagine taking a message and setting it as the x coordinate, and solving for y to get a point on the curve. It is slightly more complicated than this in practice, but this is the general idea.

![real_ec](pictures/image01.gif)

### The Downside

It is not all roses in the world of elliptic curves; there have been some questions and uncertainties that have held them back from being fully embraced by everyone in the industry.

One point that has been in the news recently is the Dual Elliptic Curve Deterministic Random Bit Generator [Dual_EC_DRBG](https://blog.cloudflare.com/how-the-nsa-may-have-put-a-backdoor-in-rsas-cryptography-a-technical-primer). This is a random number generator standardized by the **National Institute of Standards and Technology (NIST)**, and promoted by the NSA. **Dual_EC_DRBG** generates random-looking numbers using the mathematics of elliptic curves. The algorithm itself involves taking points on a curve and repeatedly performing an elliptic curve "dot" operation. After publication, it was reported that it could have been designed with a backdoor, meaning that the sequence of numbers returned could be fully predicted by someone with the right secret number. Recently, the company RSA recalled several of their products because this random number generator was set as the default PRNG for their line of security products. Whether or not this random number generator was written with a backdoor does not change the strength of the elliptic curve technology itself, but it does raise questions about the standardization process for elliptic curves. As we've written about before, it's also part of the reason that attention should be spent ensuring that your system is using adequately random numbers. In a future blog post, we will go into how a backdoor could be snuck into the specification of this algorithm.

Some of the more skeptical cryptographers in the world now have a general distrust for NIST itself and the standards it has published that were supported by the NSA. Almost all of the widely implemented elliptic curves fall into this category. There are no known attacks on these special curves, chosen for their efficient arithmetic; however, bad curves do exist, and some feel it is better to be safe than sorry. There has been progress in developing curves with efficient arithmetic outside of **NIST**, including **curve 25519** created by Daniel Bernstein (djb) and more recently computed curves by Paulo Baretto and collaborators, though widespread adoption of these curves are several years away. Until these non-traditional curves are implemented by browsers, they won't be able to be used for securing cryptographic transport on the web.

Another uncertainty about elliptic curve cryptography is related to patents. There are over **130 patents** that cover specific uses of elliptic curves owned by BlackBerry (through their 2009 acquisition of Certicom). Many of these patents were licensed for use by private organizations and even the NSA. This has given some developers pause over whether their implementations of ECC infringe upon this patent portfolio. In 2007, Certicom filed suit against Sony for some uses of elliptic curves; however, that lawsuit was dismissed in 2009. There are now many implementations of elliptic curve cryptography that are thought to not infringe upon these patents and are in wide use.

The ECDSA digital signature has a drawback compared to RSA in that it requires a good source of entropy. Without proper randomness, the private key could be revealed. A flaw in the random number generator on Android allowed hackers to find the ECDSA private key used to protect the bitcoin wallets of several people in early 2013. Sony's Playstation implementation of ECDSA had a similar vulnerability. A good source of random numbers is needed on the machine making the signatures. Dual_EC_DRBG is not recommended.


## Hidden Threats of NSA

### Algorithms and Mathematics vs. People

We've discussed algorithms and mathematics, but now let's address the human element, which complicates things.

Previously, we mentioned weak elliptic curves and added random seeds to domain parameters for security. Standard NIST curves appear verifiably random. However, the origin of these random seeds is unknown, raising suspicion about their integrity.

### Concerns about NIST Curves

Random seeds for NIST curves lack justification, unlike those for MD5, Blowfish, and RC5. This uncertainty opens the possibility that NIST might have standardized weak elliptic curves, similar to their standardization of a vulnerable random number generator.

### Security Implications

"Verifiably random" doesn't equate to "secure." If algorithms are compromised, no key length can protect us. RSA, which doesn't require special domain parameters, might be a safer alternative if we can't trust authorities.

### Example in Use

TLS connections, like those to [Google](https://google.com), often use NIST curves, such as prime256v1 (secp256p1).
